In [1]:
import os

# 대상 디렉터리
label_dir = r"C:\kfood_YOLO_dataset\train\labels"

# 파일 이름이 'Img_129_'로 시작하고 '.txt'로 끝나는 경우만 처리
for filename in os.listdir(label_dir):
    if filename.startswith("Img_129_") and filename.endswith(".txt"):
        file_path = os.path.join(label_dir, filename)
        
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        updated_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] == '90':  # 클래스 ID가 90이면
                parts[0] = '129'            # 129로 교체
            updated_lines.append(' '.join(parts))

        # 변경된 내용을 다시 파일에 저장
        with open(file_path, 'w') as file:
            file.write('\n'.join(updated_lines) + '\n')

print("변경 완료되었습니다.")


변경 완료되었습니다.


In [2]:
import cv2

# 사용자가 지정한 이미지 파일 목록
image_files = [
    "baekban.jpg",
    "chicken.png",
    "dakgalbi.png",
    "baekban_result.jpg"
]

# 각 이미지의 크기 출력
for img_path in image_files:
    img = cv2.imread(img_path)
    if img is not None:
        h, w = img.shape[:2]
        print(f"{img_path}: {w} x {h}")
    else:
        print(f"{img_path}: 파일을 열 수 없습니다.")


baekban.jpg: 675 x 506
chicken.png: 1350 x 1400
dakgalbi.png: 1400 x 1050
baekban_result.jpg: 675 x 506


## Test

In [17]:
from ultralytics import YOLO

model = YOLO("best.pt")

results = model.predict(
    source="Img_069_0962.jpg", 
    conf=0.1, 
    save=True,          # 🔥 결과 이미지 저장
    save_txt=True,      # (선택) 텍스트 파일도 저장
    project="runs/detect", 
    name="predict_result"
)


image 1/1 c:\Python311\Python Tutorial\supplement_recommend\Img_069_0962.jpg: 640x416 7 Gimbaps, 116.4ms
Speed: 2.4ms preprocess, 116.4ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 416)
Results saved to runs\detect\predict_result15
1 label saved to runs\detect\predict_result15\labels


## detect.py

In [ ]:
# detect.py

from ultralytics import YOLO
import cv2
import os
import json

# Model Path
MODEL_PATH = "runs/train/yolo11s_epoch50/weights/best.pt"

# Label List (ENG)
LABELS_ENG = [
    "Galbi-gui", "Galchi-gui", "Godeungeo-gui", "Gopchang-gui", "Dakgalbi", "Deodeok-gui", "Tteokgalbi", "Bulgogi", "Samgyeopsal", "Jangeo-gui",
    "Joge-gui", "Jogi-gui", "Hwangtae-gui", "Hoonjeo-ori", "Gyeran-guk", "Tteokguk_Mandu-guk", "Mu-guk", "Miyeok-guk", "Bugeo-guk", "Siraegi-guk",
    "Yukgaejang", "Kongnamul-guk", "Gwamegi", "Yangnyeom-chicken", "Jeotgal", "Kongjaban", "Pyeonyuk", "Pizza", "Fried-chicken", "Gat-kimchi",
    "Kkakdugi", "Nabak-kimchi", "Musengchae", "Baechu-kimchi", "Baek-kimchi", "Buchu-kimchi", "Yeolmu-kimchi", "Oi-sobagi", "Chonggak-kimchi", "Pa-kimchi",
    "Gaji-bokkeum", "Gosari-namul", "Miyeokjulgi-bokkeum", "Sokju-namul", "Sigeumchi-namul", "Aehobak-bokkeum", "Gyeongdan", "Kkul-tteok", "Songpyeon", "Mandu",
    "Ramyeon", "Mak-guksu", "Mul-naengmyeon", "Bibim-naengmyeon", "Sujebi", "Yeolmu-guksu", "Janchi-guksu", "Jjajangmyeon", "Jjamppong", "Jjolmyeon",
    "Kalguksu", "Kong-guksu", "Ghwari-gochu-muchim", "Doraji-muchim", "Dotorimuk", "Japchae", "Kongnamul-muchim", "Hongeo-muchim", "Hoe-muchim", "Gimbap",
    "Kimchi-bokkeum-bap", "Nurungji", "Bibimbap", "Saeu-bokkeum-bap", "Albap", "Yubu-chobap", "Japgok-bap", "Jumeok-bap", "Gamja-chae-bokkeum", "Geon-saeu-bokkeum",
    "Gochujang-jinmichae-bokkeum", "Dubu-kimchi", "Tteokbokki", "Rabokki", "Myeolchi-bokkeum", "Soseji-bokkeum", "Eomuk-bokkeum", "Ojingeo-chae-bokkeum", "Jeyuk-bokkeum", "Jjuggumi-bokkeum",
    "Bossam", "Sujeonggwa", "Sikhye", "Ganjang-gejang", "Yangnyeom-gejang", "Ggaetnip-jangajji", "Tteokkochi", "Gamja-jeon", "Gyeran-mari", "Gyeran-fry",
    "Kimchi-jeon", "Donggeurangttaeng", "Saengseon-jeon", "Pa-jeon", "Hobak-jeon", "Gopchang-jeongol", "Galchi-jorim", "Gamja-jorim", "Godeungeo-jorim", "Ggongechi-jorim",
    "Dubu-jorim", "Dangkong-jorim", "Maechurial-jang-jorim", "Yeongeun-jorim", "Ueong-jorim", "Jang-jorim", "Kodari-jorim", "Jeonbok-juk", "Hobak-juk", "Kimchi-jjigae",
    "Dakgaejang", "Dongtae-jjigae", "Doenjang-jjigae", "Sundubu-jjigae", "Galbi-jjim", "Gyeran-jjim", "Kimchi-jjim", "Ggomak-jjim", "Dak-bokkeum-tang", "Suyuk",
    "Sundae", "Jokbal", "Jjimdak", "Haemul-jjim", "Galbi-tang", "Gamja-tang", "Gomtang_Seolleongtang", "Maeun-tang", "Samgye-tang", "Chueo-tang",
    "Gochu-twigim", "Saeu-twigim", "Ojingeo-twigim", "Yakgwa", "Yaksik", "Hangwa", "Meongge", "Sannakji", "Mulhoe", "Yukhoe"
]

# Label List (KOR)
LABELS_KOR = [
    "갈비구이", "갈치구이", "고등어구이", "곱창구이", "닭갈비", "더덕구이", "떡갈비", "불고기", "삼겹살", "장어구이",
    "조개구이", "조기구이", "황태구이", "훈제오리", "계란국", "떡국/만둣국", "무국", "미역국", "북엇국", "시래기국",
    "육개장", "콩나물국", "과메기", "양념치킨", "젓갈", "콩자반", "편육", "피자", "후라이드치킨", "갓김치",
    "깍두기", "나박김치", "무생채", "배추김치", "백김치", "부추김치", "열무김치", "오이소박이", "총각김치", "파김치",
    "가지볶음", "고사리나물", "미역줄기볶음", "숙주나물", "시금치나물", "애호박볶음", "경단", "꿀떡", "송편", "만두",
    "라면", "막국수", "물냉면", "비빔냉면", "수제비", "열무국수", "잔치국수", "짜장면", "짬뽕", "쫄면",
    "칼국수", "콩국수", "꽈리고추무침", "도라지무침", "도토리묵", "잡채", "콩나물무침", "홍어무침", "회무침", "김밥",
    "김치볶음밥", "누룽지", "비빔밥", "새우볶음밥", "알밥", "유부초밥", "잡곡밥", "주먹밥", "감자채볶음", "건새우볶음",
    "고추장진미채볶음", "두부김치", "떡볶이", "라볶이", "멸치볶음", "소시지볶음", "어묵볶음", "오징어채볶음", "제육볶음", "쭈꾸미볶음",
    "보쌈", "수정과", "식혜", "간장게장", "양념게장", "깻잎장아찌", "떡꼬치", "감자전", "계란말이", "계란후라이",
    "김치전", "동그랑땡", "생선전", "파전", "호박전", "곱창전골", "갈치조림", "감자조림", "고등어조림", "꽁치조림",
    "두부조림", "땅콩조림", "메추리알장조림", "연근조림", "우엉조림", "장조림", "코다리조림", "전복죽", "호박죽", "김치찌개",
    "닭개장", "동태찌개", "된장찌개", "순두부찌개", "갈비찜", "계란찜", "김치찜", "꼬막찜", "닭볶음탕", "수육",
    "순대", "족발", "찜닭", "해물찜", "갈비탕", "감자탕", "곰탕/설렁탕", "매운탕", "삼계탕", "추어탕",
    "고추튀김", "새우튀김", "오징어튀김", "약과", "약식", "한과", "멍게", "산낙지", "물회", "육회"
]

MODEL_PATH = "runs/train/yolo11s_epoch50/weights/best.pt"
model = YOLO(MODEL_PATH)  # 모델 한 번만 로딩

def run_detection(img_path, save_path=None, conf_thres=0.5):
    results = model(img_path, conf=conf_thres)[0]
    img = cv2.imread(img_path)
    detected = []

    for box in results.boxes:
        cls_id = int(box.cls[0])
        score = float(box.conf[0])
        label_eng = LABELS_ENG[cls_id]
        label_kor = LABELS_KOR[cls_id]

        if save_path:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 2)
            cv2.putText(img, f"{label_kor} {score:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)

        detected.append({
            "class_index": cls_id,
            "label_eng": label_eng,
            "label_kor": label_kor,
            "confidence": round(score, 4)
        })

    if save_path:
        cv2.imwrite(save_path, img)
    return detected

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument('--img', type=str, required=True, help='Input image path')
    parser.add_argument('--conf', type=float, default=0.5, help='Confidence threshold')
    args = parser.parse_args()

    result = run_detection(args.img, conf_thres=args.conf)
    print(json.dumps(result, ensure_ascii=False, indent=2))


## main.py

In [ ]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse, FileResponse
from detect import run_detection
import shutil
import os
from uuid import uuid4

app = FastAPI()

UPLOAD_DIR = "uploads"
RESULT_DIR = "results"

# 디렉토리 준비
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

@app.post("/detect")
async def detect_food(file: UploadFile = File(...)):
    # 저장할 고유 파일명 생성
    file_id = str(uuid4())
    upload_path = os.path.join(UPLOAD_DIR, f"{file_id}.jpg")
    result_path = os.path.join(RESULT_DIR, f"{file_id}_result.jpg")

    # 업로드 파일 저장
    with open(upload_path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    # 탐지 수행
    results = run_detection(image_path=upload_path, save_path=result_path)

    return JSONResponse(content={
        "results": results,
        "result_image": f"/result_image/{file_id}"
    })

@app.get("/result_image/{file_id}")
async def get_result_image(file_id: str):
    path = os.path.join(RESULT_DIR, f"{file_id}_result.jpg")
    if not os.path.exists(path):
        return JSONResponse(content={"error": "Not found"}, status_code=404)
    return FileResponse(path)


In [1]:
import os
os.getcwd()

'c:\\Python311\\Python Tutorial\\supplement_recommend'

In [2]:
import os
from collections import defaultdict

# 라벨 파일이 있는 폴더 경로
label_dir = r"C:\kfood_YOLO_dataset\train\labels"

# 클래스별 개수를 저장할 딕셔너리

class_counts = defaultdict(int)

# 폴더 내 파일 이름 탐색
for filename in os.listdir(label_dir):
    if filename.startswith("Img_"):
#     if filename.startswith("Img_") and filename.endswith(".txt"):  # 파일 이름이 "Img_"로 시작하고 ".txt"로 끝나는 경우
        class_id = filename.split("_")[1]  # "Img_XXX_YYYY.txt"에서 XXX 추출
        class_counts[class_id] += 1

# 결과 출력
for class_id, count in sorted(class_counts.items(), key=lambda x: int(x[0])):  # 클래스 ID 기준 정렬
    print(f"클래스 {class_id}: {count}개")

클래스 000: 741개
클래스 001: 747개
클래스 002: 726개
클래스 003: 665개
클래스 004: 498개
클래스 005: 523개
클래스 006: 528개
클래스 007: 451개
클래스 008: 561개
클래스 009: 465개
클래스 010: 649개
클래스 011: 540개
클래스 012: 390개
클래스 013: 476개
클래스 014: 410개
클래스 015: 396개
클래스 016: 430개
클래스 017: 543개
클래스 018: 204개
클래스 019: 457개
클래스 020: 447개
클래스 021: 485개
클래스 022: 548개
클래스 023: 608개
클래스 024: 412개
클래스 025: 597개
클래스 026: 319개
클래스 027: 524개
클래스 028: 574개
클래스 029: 286개
클래스 030: 630개
클래스 031: 598개
클래스 032: 700개
클래스 033: 619개
클래스 034: 587개
클래스 035: 323개
클래스 036: 428개
클래스 037: 695개
클래스 038: 709개
클래스 039: 641개
클래스 040: 497개
클래스 041: 628개
클래스 042: 573개
클래스 043: 551개
클래스 044: 604개
클래스 045: 576개
클래스 046: 383개
클래스 047: 412개
클래스 048: 118개
클래스 049: 521개
클래스 050: 474개
클래스 051: 379개
클래스 052: 191개
클래스 053: 236개
클래스 054: 191개
클래스 055: 761개
클래스 056: 740개
클래스 057: 748개
클래스 058: 775개
클래스 059: 798개
클래스 060: 754개
클래스 061: 773개
클래스 062: 652개
클래스 063: 748개
클래스 064: 768개
클래스 065: 733개
클래스 066: 718개
클래스 067: 772개
클래스 068: 490개
클래스 069: 748개
클래스 070: 407개
클래스 07

In [13]:
import pyautogui

pyautogui.moveTo(751, 589)

In [ ]:
import pyautogui
import time

# 라벨 범위 설정 (005 ~ 149)
start_label = 92
end_label = 122

# 좌표 설정 (수정된 좌표)
first_position = (1108, 318)  # 첫 번째 커서 이동 위치
click_position1 = (1123, 363)  # 첫 번째 클릭

click_position2 = (516, 89)  # 두 번째 클릭
input_position = (385, 555)  # 숫자 입력할 위치
select_all_position = (304, 235)  # Ctrl + A 누를 위치
final_position = (751, 589)  # 마지막 클릭 위치

# 딜레이 설정
delay = 3  # 첫 이동 후 대기 시간 (초)
input_delay = 2  # 각 동작 간 딜레이
wait_time = 440 # 15분(900초) 대기 시간

for i in range(start_label, end_label + 1):
    label = f"{i}"  # 005 ~ 149 숫자 포맷 맞추기

    print(f"📌 {label} 처리 시작...")
    time.sleep(delay)
    # 첫 번째 커서 이동 후 대기
    pyautogui.moveTo(first_position)
    time.sleep(delay)

    # 첫 번째 클릭
    pyautogui.click(click_position1)
    time.sleep(input_delay)

    # 두 번째 클릭
    pyautogui.click(click_position2)
    time.sleep(input_delay)

    # 숫자 입력
    pyautogui.click(input_position)
    time.sleep(input_delay)
    pyautogui.write(label)  # 005 ~ 149 입력
    pyautogui.press('enter')  # Enter 입력
    time.sleep(input_delay)

    # Ctrl + A 입력
    pyautogui.click(select_all_position)
    time.sleep(input_delay)
    pyautogui.hotkey('ctrl', 'a')  # Ctrl + A
    time.sleep(input_delay)

    # 마지막 위치 클릭
    pyautogui.click(final_position)
    time.sleep(input_delay)

    print(f"✅ {label} 처리 완료! 15분 대기 중...")
    
    if i < end_label:
        time.sleep(wait_time)  # 15분 대기

print("🎉 모든 매크로 작업 완료!")


📌 92 처리 시작...
✅ 92 처리 완료! 15분 대기 중...
📌 93 처리 시작...
✅ 93 처리 완료! 15분 대기 중...
📌 94 처리 시작...
✅ 94 처리 완료! 15분 대기 중...
📌 95 처리 시작...
✅ 95 처리 완료! 15분 대기 중...
📌 96 처리 시작...
✅ 96 처리 완료! 15분 대기 중...
📌 97 처리 시작...
✅ 97 처리 완료! 15분 대기 중...


In [18]:
import os
import shutil

# 기존 YOLO 데이터셋 경로
old_dataset_path = "C:/kfood_YOLO_dataset"
new_dataset_path = "C:/kfood_YOLO_fixed"

# 새로운 YOLO 형식 폴더 생성
for split in ["train", "val", "test"]:
    os.makedirs(f"{new_dataset_path}/{split}/images", exist_ok=True)
    os.makedirs(f"{new_dataset_path}/{split}/labels", exist_ok=True)

# 기존 폴더에서 파일 이동
for split in ["train", "val", "test"]:
    old_images_path = f"{old_dataset_path}/images/{split}"
    old_labels_path = f"{old_dataset_path}/labels/{split}"
    
    new_images_path = f"{new_dataset_path}/{split}/images"
    new_labels_path = f"{new_dataset_path}/{split}/labels"

    # 이미지 파일 이동
    for file_name in os.listdir(old_images_path):
        shutil.move(os.path.join(old_images_path, file_name), os.path.join(new_images_path, file_name))

    # 라벨 파일 이동
    for file_name in os.listdir(old_labels_path):
        shutil.move(os.path.join(old_labels_path, file_name), os.path.join(new_labels_path, file_name))

print("✅ YOLO 폴더 구조 수정 완료!")


✅ YOLO 폴더 구조 수정 완료!


In [7]:
import os

# 폴더 경로 설정
image_dir = r"C:\kfood_YOLO_dataset\train\images"
label_dir = r"C:\kfood_YOLO_dataset\train\labels"

# 이미지/라벨 파일의 이름(확장자 제외한 이름) 수집
image_files = {os.path.splitext(f)[0] for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))}
label_files = {os.path.splitext(f)[0] for f in os.listdir(label_dir) if os.path.isfile(os.path.join(label_dir, f))}

# 비교 결과
only_images = sorted(image_files - label_files)
only_labels = sorted(label_files - image_files)

# 요약 출력
print(f"이미지 총 개수: {len(image_files)}")
print(f"라벨 총 개수 : {len(label_files)}")
print(f"라벨 없는 이미지: {len(only_images)}개")
print(f"이미지 없는 라벨: {len(only_labels)}개\n")

# 누락된 파일 이름 일부 출력
if only_images:
    print("✅ 라벨이 없는 이미지 파일들 (일부):")
    print("\n".join(only_images[:10]))
if only_labels:
    print("\n⚠️ 이미지가 없는 라벨 파일들 (일부):")
    print("\n".join(only_labels[:10]))

이미지 총 개수: 85740
라벨 총 개수 : 85740
라벨 없는 이미지: 0개
이미지 없는 라벨: 0개



In [6]:
import os
from collections import defaultdict

# 라벨 파일이 있는 폴더 경로
label_dir = r"C:\kfood_YOLO_dataset\train\images"

# 클래스별 개수를 저장할 딕셔너리
class_counts = defaultdict(int)

# 폴더 내 파일 이름 탐색
for filename in os.listdir(label_dir):
    if filename.startswith("Img_") and filename.endswith(".jpg"):  # 파일 이름이 "Img_"로 시작하고 ".txt"로 끝나는 경우
        class_id = filename.split("_")[1]  # "Img_XXX_YYYY.txt"에서 XXX 추출
        class_counts[class_id] += 1

# 결과 출력
for class_id, count in sorted(class_counts.items(), key=lambda x: int(x[0])):  # 클래스 ID 기준 정렬
    print(f"클래스 {class_id}: {count}개")

클래스 000: 741개
클래스 001: 747개
클래스 002: 726개
클래스 003: 665개
클래스 004: 498개
클래스 005: 523개
클래스 006: 528개
클래스 007: 451개
클래스 008: 561개
클래스 009: 465개
클래스 010: 649개
클래스 011: 540개
클래스 012: 390개
클래스 013: 476개
클래스 014: 410개
클래스 015: 396개
클래스 016: 430개
클래스 017: 543개
클래스 018: 204개
클래스 019: 457개
클래스 020: 447개
클래스 021: 485개
클래스 022: 548개
클래스 023: 608개
클래스 024: 412개
클래스 025: 597개
클래스 026: 319개
클래스 027: 524개
클래스 028: 574개
클래스 029: 286개
클래스 030: 630개
클래스 031: 598개
클래스 032: 700개
클래스 033: 619개
클래스 034: 587개
클래스 035: 323개
클래스 036: 428개
클래스 037: 695개
클래스 038: 709개
클래스 039: 641개
클래스 040: 497개
클래스 041: 628개
클래스 042: 573개
클래스 043: 551개
클래스 044: 604개
클래스 045: 576개
클래스 046: 383개
클래스 047: 412개
클래스 048: 118개
클래스 049: 521개
클래스 050: 474개
클래스 051: 379개
클래스 052: 191개
클래스 053: 236개
클래스 054: 191개
클래스 055: 761개
클래스 056: 740개
클래스 057: 748개
클래스 058: 775개
클래스 059: 798개
클래스 060: 754개
클래스 061: 773개
클래스 062: 652개
클래스 063: 748개
클래스 064: 768개
클래스 065: 733개
클래스 066: 718개
클래스 067: 772개
클래스 068: 490개
클래스 069: 748개
클래스 070: 407개
클래스 07

# class 폴더로 나누기

In [2]:
import os
import shutil

# 원본 라벨 파일이 있는 폴더
label_dir = r"C:\kfood_YOLO_dataset\train\labels"
# 타겟 폴더 (클래스별 폴더를 생성할 경로)
target_root = r"C:\kfood_YOLO_dataset\train"

# 복사 대상 클래스 번호 범위
start_class = 123
end_class = 143

# labels 폴더 내 파일 탐색
for filename in os.listdir(label_dir):
    if filename.endswith(".txt") and filename.startswith("Img_"):
        try:
            # 파일명에서 클래스 번호 추출
            class_num = int(filename.split('_')[1])
            if start_class <= class_num <= end_class:
                class_folder = os.path.join(target_root, str(class_num))

                # 클래스 폴더가 없으면 생성
                os.makedirs(class_folder, exist_ok=True)

                # 원본 파일 경로
                src_path = os.path.join(label_dir, filename)
                # 복사될 파일 경로
                dst_path = os.path.join(class_folder, filename)

                # 파일 복사
                shutil.copy2(src_path, dst_path)
                print(f"복사됨: {src_path} -> {dst_path}")
        except ValueError:
            continue  # class_num 추출 실패 시 무시

print("파일 복사 완료.")


복사됨: C:\kfood_YOLO_dataset\train\labels\Img_123_0003.txt -> C:\kfood_YOLO_dataset\train\123\Img_123_0003.txt
복사됨: C:\kfood_YOLO_dataset\train\labels\Img_123_0014.txt -> C:\kfood_YOLO_dataset\train\123\Img_123_0014.txt
복사됨: C:\kfood_YOLO_dataset\train\labels\Img_123_0020.txt -> C:\kfood_YOLO_dataset\train\123\Img_123_0020.txt
복사됨: C:\kfood_YOLO_dataset\train\labels\Img_123_0025.txt -> C:\kfood_YOLO_dataset\train\123\Img_123_0025.txt
복사됨: C:\kfood_YOLO_dataset\train\labels\Img_123_0026.txt -> C:\kfood_YOLO_dataset\train\123\Img_123_0026.txt
복사됨: C:\kfood_YOLO_dataset\train\labels\Img_123_0029.txt -> C:\kfood_YOLO_dataset\train\123\Img_123_0029.txt
복사됨: C:\kfood_YOLO_dataset\train\labels\Img_123_0030.txt -> C:\kfood_YOLO_dataset\train\123\Img_123_0030.txt
복사됨: C:\kfood_YOLO_dataset\train\labels\Img_123_0032.txt -> C:\kfood_YOLO_dataset\train\123\Img_123_0032.txt
복사됨: C:\kfood_YOLO_dataset\train\labels\Img_123_0042.txt -> C:\kfood_YOLO_dataset\train\123\Img_123_0042.txt
복사됨: C:\kfood_YOLO_

In [4]:
import os

# 이미지 및 라벨 폴더 경로
image_folder = r"C:\kfood_YOLO_dataset\train\images"
label_folder = r"C:\kfood_YOLO_dataset\train\labels"

# 기준이 될 마지막 파일명
max_filename = "Img_149_0999"

# 이미지 파일 목록 가져오기 (확장자 포함)
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

# 라벨 파일 목록 가져오기 (확장자 제거)
label_files = {os.path.splitext(f)[0] for f in os.listdir(label_folder) if f.endswith(".txt")}

# 삭제할 이미지 리스트 생성 (파일명이 max_filename 이하 & 라벨이 없는 경우)
images_to_delete = [
    f for f in image_files 
    if os.path.splitext(f)[0] not in label_files and os.path.splitext(f)[0] <= max_filename
]

# 이미지 삭제 (안전 확인 후 진행)
for img in images_to_delete:
    img_path = os.path.join(image_folder, img)
    os.remove(img_path)
    print(f"삭제됨: {img_path}")

print(f"\n총 {len(images_to_delete)}개의 이미지 파일이 삭제되었습니다.")


총 0개의 이미지 파일이 삭제되었습니다.


In [3]:
import os

# 이미지 및 라벨 폴더 경로
image_folder = r"C:\kfood_YOLO_dataset\train\images"
label_folder = r"C:\kfood_YOLO_dataset\train\labels"

# 기준이 될 마지막 파일명
max_filename = "Img_149_0999"

# 이미지 파일 목록 가져오기 (확장자 제거)
image_files = {os.path.splitext(f)[0] for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))}

# 라벨 파일 목록 가져오기 (확장자 포함)
label_files = [f for f in os.listdir(label_folder) if f.endswith(".txt")]

# 삭제할 라벨 리스트 생성 (파일명이 max_filename 이하 & 이미지가 없는 경우)
labels_to_delete = [
    f for f in label_files 
    if os.path.splitext(f)[0] not in image_files and os.path.splitext(f)[0] <= max_filename
]

# 라벨 삭제 (안전 확인 후 진행)
for label in labels_to_delete:
    label_path = os.path.join(label_folder, label)
    os.remove(label_path)
    print(f"삭제됨: {label_path}")

print(f"\n총 {len(labels_to_delete)}개의 라벨 파일이 삭제되었습니다.")



총 0개의 라벨 파일이 삭제되었습니다.
